# Developing REST APIs with Amazon API Gateway

## Overview and objectives
In this project, I will create a REST application programming interface (API) by using Amazon API Gateway.

After reading this document, you also should be able to:

- Create simple mock endpoints for REST APIs and use them in your website.
- Enable Cross-Origin Resource Sharing (CORS)

## Scenario
In the previous lab, I took on the role of Sofía to build a web application for the café. As part of this process, I created an Amazon DynamoDB table that was named FoodProducts, where I stored information about café menu items.

I then loaded data that was formatted in JavaScript Object Notation (JSON) into the database table. The table structure looked similar to the following table (one line item of table data is shown as an example):

| product_name | description | price_in_cents |product_id | tags3 | special |
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
|  apple pie slice   |  A delicious slice of Frank's homemade pie.   |  595   |  a444   |  [ { "S" : "pie slice" }, { "S" : "on offer" } ]   |  1   |

In the previous lab I also configured code that used the AWS SDK for Python (Boto3) to:

- Scan a DynamoDB table to retrieve product details.
- Return a single item by product name using get-item as a proof of concept
- Create a Global Secondary Index (GSI) called special_GSI that you could use to filter out menu items that are on offer and not out of stock.

In this project, I will continue the role of Sofía. I will use Amazon API Gateway to configure mock data endpoints. There are three that I will create:

- [GET] /products (which will eventually invoke a DynamoDB table scan)
- [GET] /products/on_offer (which will eventually invoke a DynamoDB index scan and filter)
- [POST] /create_report (which will eventually trigger a batch process that will send out a report)

Then in the [project that follows this one](), I will replace the mock endpoints with real endpoints, so that the web application can connect to the DynamoDB backend.

When you start the AWS simulation, the following resources are pre-created in the account.

![start-arch](images/start-arch.png)

However, by the end of this project, I have created the following architecture:

![end-arch](images/end-arch.png)



# Task 1: Preparing the development environment

In this first task, I will configure your AWS Cloud9 environment so that you can create the REST API.

From the AWS Management Console, connect to the AWS Cloud9 IDE named Cloud9 Instance.
 
![cloud9-instance](images/cloud9-instance.png)

Download and extract the files that you will need for this lab.

In the same terminal, run the following command:

In [ ]:
wget https://aws-tc-largeobjects.s3.us-west-2.amazonaws.com/CUR-TF-200-ACCDEV-2-91558/04-lab-api/code.zip -P /home/ec2-user/environment

Notice that a code.zip file was downloaded to the AWS Cloud9 instance. The file is listed in the Environment window.

![restcodezip](images/restcodezip.png)

Extract the file:

In [ ]:
unzip code.zip

![restunzip](images/restunzip.png)

Run the script that upgrades the versions of Python and the AWS CLI installed in your IDE environment,  and also creates the cafe website in your AWS account.

In [ ]:
chmod +x resources/setup.sh && resources/setup.sh

The script will prompt you for the IP address by which your computer is known to the internet.

![rest-chmod](images/rest-chmod.png)

Use www.whatismyip.com to discover this address and then paste the IPv4 address into the command prompt and finish running the script.

### Verify the version of AWS CLI installed.

In the AWS Cloud9 Bash terminal (at the bottom of the IDE), run the following command:

In [ ]:
aws --version

In [ ]:
aws-cli/2.22.34 Python/3.12.6 Linux/5.10.230.223.885.amzn2.x86_64.exe

The output should indicate that version 2 is installed.

### Verify that the SDK for Python is installed.

Run the following command:

In [ ]:
pip show boto3

![rest-boto](images/rest-boto.png)

Note: If you see a message about not using the latest version of pip, ignore the message.

### Verify that the cafe website can be loaded in a browser tab.

Load the website in a browser tab.

In a browser tab, open the Amazon S3 console.

Choose your bucket name, and then choose Objects.

If the files that the script just uploaded do not display, choose the refresh icon to view them.

Choose the index.html file.

Copy the Object URL. It will be in the following format.

In [ ]:
https://<bucket-name>.s3.amazon.com/index.html

![rest-index](images/rest-index.png)

Verify that the website displays by pasting the full URL into your browser.

![rest-cafe](images/rest-cafe.png)

### Note particular website details.

Notice in the Browse Pastries section that there are two buttons.

The "on offer" view displays by default and it shows six menu items.

Select the "view all" view. Notice that many more menu items display.

![cafe-onoffer-viewall](images/cafe-onoffer-viewall.png)

Optionally, expose the developer console view in your browser.

For example, if you are using Chrome, choose View > Developer > View JavaScript Console
If you are using Firefox, choose Tools > Web Developer > Web Console.
Analysis: These menu details that display on the website are currently being read out of the all_products_on_offer.json and all_products.json files that are hosted in your S3 bucket. You also have a copy of these same files in your Cloud9 IDE, in the resources/website directory.

If you are looking at the browser developer console view details, you will also see a log entry written by the main.js file, indicating that hardcoded data is being used.  Later in the lab, you will see that the number of menu items that display changes and that the messages in the console change as well.

Leave the website open in this browser tab, you will return to it towards the end of the lab.